# 🔢 **Symbolic Regression - Analysis of equations**  

For this analysis, will be consider the file "optimization_sr.db" which was made by the pilot version of the script_sr.py, which considered the binary and unary operators as a parameter that should be optimized. In the new version, the operarors are considered as a default parameter in Symbolic Regression (SR) model. Here, the objective is to visualize the expressions obtained by SR with the optimized parameters.

***

### 📚 **Importing libraries**

In [1]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.cluster import BisectingKMeans

from script_sr import Clustering_SR, cross_validation

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


### 📂 **Importing data and split**

In [2]:
data = pd.read_csv("../data/train.csv")

In [3]:
X = data.drop(columns=["critical_temp"])
y = data["critical_temp"]

X_test, X_train, y_test, y_train = train_test_split(
        X, y, test_size=0.9, random_state=1702
    )
y_test = np.clip(y_test, 1e-6, None)
y_train = np.clip(y_train, 1e-6, None)

### 📔 **Importing study and its params**

In [4]:
study_name = "optimization_clusters_gam_teste15_07_2025_15_40_36" 
# The first version of script_sr.py defined the name with "gam" word. This was fixed in the new version!
from pathlib import Path

storage_url = f"sqlite:///../Optuna_files/optimization_sr.db"


study_sr = optuna.load_study(study_name=study_name, storage=storage_url)

In [5]:
print(f"The best params were: {study_sr.best_params}")

The best params were: {'clusterer': 'bisecting_kmeans', 'n_clusters_bkmeans': 17, 'bisecting_strategy': 'largest_cluster', 'n_iterations': 23, 'maxsize': 29, 'maxdepth': 14, 'n_binary_operators': 9, 'binary_operators_0': '*', 'binary_operators_1': '^', 'binary_operators_2': '*', 'binary_operators_3': '*', 'binary_operators_4': '+', 'binary_operators_5': '*', 'binary_operators_6': '+', 'binary_operators_7': '+', 'binary_operators_8': '-', 'n_unary_operators': 19, 'unary_operators_0': 'sin', 'unary_operators_1': 'sin', 'unary_operators_2': 'neg', 'unary_operators_3': 'tan', 'unary_operators_4': 'cube', 'unary_operators_5': 'cos', 'unary_operators_6': 'sqrt', 'unary_operators_7': 'inv', 'unary_operators_8': 'sin', 'unary_operators_9': 'tan', 'unary_operators_10': 'abs', 'unary_operators_11': 'exp', 'unary_operators_12': 'exp', 'unary_operators_13': 'log', 'unary_operators_14': 'log', 'unary_operators_15': 'sqrt', 'unary_operators_16': 'abs', 'unary_operators_17': 'neg', 'unary_operators_1

### 🦾 **Training the model**

In [6]:
clusterer = BisectingKMeans(n_clusters=17,
                            bisecting_strategy="largest_cluster",)

In [7]:
model = Clustering_SR(clusterer=clusterer, 
              n_iterations=23, 
              maxsize=29, 
              maxdepth=14,
              select_k_features=20)

In [8]:
model.fit(X_train, y_train)

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Using features ['entropy_atomic_mass' 'mean_fie' 'range_fie' 'std_fie' 'wtd_std_fie'
 'wtd_range_atomic_radius' 'wtd_std_atomic_radius' 'wtd_mean_FusionHeat'
 'entropy_FusionHeat' 'range_FusionHeat' 'wtd_std_FusionHeat'
 'mean_ThermalConductivity' 'wtd_gmean_ThermalConductivity'
 'entropy_ThermalConductivity' 'wtd_entropy_ThermalConductivity'
 'std_ThermalConductivity' 'wtd_mean_Valence' 'wtd_gmean_Valence'
 'wtd_entropy_Valence' 'wtd_std_Valence']
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:96: UserWarning: You are using the `^` operator, but have not set up `constraints` for it. This may lead to overly complex expressions. One typical constraint is to use `constraints={..., '^': (-1, 1)}`, which will allow arbitrary-complexity base (-1) but only powers such as a constant o

,clusterer,BisectingKMea...n_clusters=17)
,n_iterations,23
,maxsize,29
,maxdepth,14
,select_k_features,20
,n_clusters,17
,init,'random'
,n_init,1
,random_state,None
,max_iter,300
,verbose,0


### 🔢 **Viewing equations**

#### **Cluster 1**

In [12]:
model.expressions_[0]

,complexity,loss,equation
0,1,22.927567,6.466067
1,3,21.490635,wtd_std_FusionHeat + 7.004014
2,4,19.164429,square(wtd_std_FusionHeat + 3.0578759)
3,6,18.182203,square(wtd_std_FusionHeat + 3.0578759) + entro...
4,10,18.014282,((12.306311 / exp(sqrt(abs(wtd_std_FusionHeat)...
5,12,16.226456,((0.40876484 ^ cos(exp(wtd_gmean_Valence) - sq...
6,19,16.103180,(exp(square(cos(wtd_gmean_Valence - 0.96376354...
7,21,15.511872,((0.40876484 ^ (cos(exp(wtd_gmean_Valence) - s...
8,22,15.458944,abs(entropy_ThermalConductivity + ((abs(wtd_st...


In [18]:
print(model.expressions_[0].iloc[8]["equation"])

abs(entropy_ThermalConductivity + ((abs(wtd_std_atomic_radius + (((wtd_mean_Valence - -0.7350441) / range_FusionHeat) / (inv(square(inv(wtd_mean_Valence))) + -1.3224036))) ^ 0.25417194) - -4.169578))


$$
T_c = \left| 
\mathrm{entropy\_ThermalConductivity} 
+ 
\left( 
\left| 
\mathrm{wtd\_std\_atomic\_radius} + 0.7350441
\right| 
\cdot 
\frac{1}{\mathrm{range\_FusionHeat}} 
\cdot 
\left( 
(\frac{1}{\mathrm{wtd\_mean\_Valence}}) ^ 2
- 1.3224036
\right) 
\right)^{0.25417194} 
+ 4.169578
\right|
$$

#### **Cluster 2**

In [22]:
model.expressions_[1]

,complexity,loss,equation
0,1,38.621660,5.9513054
1,2,33.488266,square(wtd_range_atomic_radius)
2,3,28.333310,wtd_mean_atomic_radius / 0.18650644
3,4,15.821792,cube(wtd_range_FusionHeat - wtd_std_fie)
4,5,15.821318,abs(cube(wtd_range_FusionHeat - wtd_std_fie))
5,6,14.611463,cube(wtd_range_FusionHeat - wtd_std_fie) - -1....
6,7,14.032359,abs(cube(wtd_range_FusionHeat - wtd_std_fie) -...
7,8,13.745005,(cube(wtd_range_FusionHeat - wtd_std_fie) - wt...
8,16,12.490854,abs(((wtd_gmean_atomic_radius + wtd_std_Fusion...
9,18,11.998098,abs(((wtd_gmean_atomic_radius + (wtd_std_Fusio...


In [24]:
print(model.expressions_[1].iloc[12]["equation"])

square(wtd_gmean_atomic_radius) + square(sin((wtd_std_FusionHeat + 0.4829266) * ((wtd_std_FusionHeat - wtd_gmean_atomic_radius) - (wtd_entropy_ThermalConductivity * wtd_gmean_atomic_radius))) - abs((mean_atomic_radius - wtd_mean_atomic_radius) - cos(wtd_range_ElectronAffinity)))


$$
T_c =
\begin{split}
& \left( \mathrm{wtd\_gmean\_atomic\_radius} \right)^2
+ \left( 
\sin\left(
\left( \mathrm{wtd\_std\_FusionHeat} + 0.4829266 \right) 
\cdot 
\left( 
\mathrm{wtd\_std\_FusionHeat} - \mathrm{wtd\_gmean\_atomic\_radius} 
- \mathrm{wtd\_entropy\_ThermalConductivity} \cdot \mathrm{wtd\_gmean\_atomic\_radius}
\right)
\right) \right. \\
& \left. \quad
- 
\left| 
\mathrm{mean\_atomic\_radius} - \mathrm{wtd\_mean\_atomic\_radius} 
- \cos\left( \mathrm{wtd\_range\_ElectronAffinity} \right)
\right|
\right)^2
\end{split}
$$



#### **Cluster 3**

In [25]:
model.expressions_[2]

,complexity,loss,equation
0,1,10.523952,4.396335
1,3,10.190161,wtd_mean_Valence + 4.154043
2,5,9.871449,cos(exp(wtd_mean_ThermalConductivity)) + 3.687...
3,6,9.734819,exp(inv(cos(cos(cos(mean_Density)))))
4,7,9.692690,(cos(exp(wtd_mean_ThermalConductivity)) + 3.25...
5,9,9.351548,exp(inv(cos(sin(mean_Density)))) + cos(exp(wtd...
6,11,9.244647,cos(exp(wtd_mean_ThermalConductivity)) + exp(i...
7,12,9.035620,(exp(inv(cos(cos(cos(mean_Density))))) + cos(e...
8,13,8.672320,abs(cos(exp(wtd_mean_ThermalConductivity)) + e...


In [27]:
print(model.expressions_[2].iloc[8]["equation"])

abs(cos(exp(wtd_mean_ThermalConductivity)) + exp(inv(cos(cos(cos(mean_Density * wtd_gmean_Valence))))))


$$
\begin{aligned}
T_c =\ & 
\left| 
\cos\left( 
\exp\left( \mathrm{wtd\_mean\_ThermalConductivity} \right) 
\right)
+ 
\exp\left( 
\frac{1}{
\cos\left( 
\cos\left( 
\cos\left( 
\mathrm{mean\_Density} \cdot \mathrm{wtd\_gmean\_Valence} 
\right)
\right)
\right)
}
\right)
\right|
\end{aligned}
$$


#### **Cluster 4**

In [28]:
model.expressions_[3]

,complexity,loss,equation
0,1,10.190784,3.2611098
1,2,9.104302,square(wtd_gmean_atomic_radius)
2,3,9.064628,wtd_gmean_atomic_radius * 2.108322
3,4,8.464461,square(wtd_mean_atomic_radius) - wtd_mean_fie
4,5,8.310187,abs(square(wtd_mean_atomic_radius) - wtd_gmean...
5,7,7.507406,abs(square(wtd_mean_atomic_radius) + wtd_std_a...
6,9,7.100302,abs(cos(wtd_gmean_atomic_radius - wtd_gmean_fi...
7,10,7.100302,abs((wtd_gmean_atomic_radius * cos(wtd_gmean_a...
8,11,7.019707,abs((wtd_gmean_atomic_radius * (cos(wtd_gmean_...
9,13,7.006529,abs(sqrt(square(1.7190316 - (wtd_gmean_atomic_...


In [29]:
print(model.expressions_[3].iloc[16]["equation"])

abs((cos(wtd_gmean_fie - (wtd_gmean_atomic_radius - cos(0.5337268 ^ (wtd_mean_atomic_radius / -2.5628972)))) * (-1.9834003 - (0.4087335 ^ cos((1.393756 ^ wtd_std_Density) / abs(range_Density))))) * wtd_gmean_atomic_radius)


$$
T_c = 
\biggl|
\begin{split}
\biggl( 
\cos\biggl(
\mathrm{wtd\_gmean\_fie} - 
\bigl(
\mathrm{wtd\_gmean\_atomic\_radius} - 
\cos\biggl(
0.5337268^{ 
\frac{
\mathrm{wtd\_mean\_atomic\_radius}
}{
-2.5628972}
} 
\biggr)
\bigr)
\biggr) \\
\cdot 
\biggl(
-1.9834003 - 
0.4087335^{ 
\cos\left(
\frac{
1.393756^{\mathrm{wtd\_std\_Density}}
}{
\bigl| \mathrm{range\_Density} \bigr|}
\right)
}
\biggr)
\biggr)
\cdot \mathrm{wtd\_gmean\_atomic\_radius}
\end{split}
\biggr|
$$





#### **Cluster 5**

In [30]:
model.expressions_[4]

,complexity,loss,equation
0,1,225.741940,16.463936
1,3,145.857250,square(square(mean_atomic_mass))
2,4,140.405850,exp(wtd_mean_atomic_mass * -1.9090296)
3,5,130.798860,(mean_atomic_mass * wtd_mean_atomic_mass) * 6....
4,6,122.854095,0.977742 / (exp(gmean_atomic_mass) ^ 1.9156873)
5,9,116.811400,abs((wtd_mean_ElectronAffinity * -8.700231) * ...
6,10,108.511154,abs((wtd_mean_fie + wtd_mean_ElectronAffinity)...
7,11,98.518670,abs(((4.7067037 - abs(wtd_mean_ThermalConducti...
8,12,95.589195,(wtd_range_FusionHeat + 2.3533518) * (((mean_a...
9,13,92.933395,abs(((wtd_range_FusionHeat + 2.5320327) * ((me...


In [31]:
print(model.expressions_[4].iloc[13]["equation"])

abs(abs(cube(wtd_std_Valence) + (cube(wtd_std_Valence) + ((wtd_range_FusionHeat + 2.3533518) * ((mean_atomic_mass * wtd_mean_ElectronAffinity) + 1.266521)))) - wtd_mean_atomic_mass)


$$
T_c = \biggl( \left(\mathrm{wtd\_std\_Valence}\right)^3 + \left(\left(\mathrm{wtd\_range\_FusionHeat} + 2.3533518\right) \cdot \left(\left(\mathrm{mean\_atomic\_mass} \cdot \mathrm{wtd\_mean\_ElectronAffinity}\right) + 1.266521\right)\right) - \mathrm{wtd\_mean\_atomic\_mass} \biggr)^3
$$


#### **Cluster 6**

In [32]:
model.expressions_[5]

,complexity,loss,equation
0,1,116.650970,11.848103
1,3,108.660355,10.871684 - wtd_gmean_ElectronAffinity
2,4,90.522740,cube(exp(cos(range_fie)))
3,6,82.210810,square((range_fie - -3.4595847) - wtd_gmean_El...
4,8,74.152100,square(mean_FusionHeat - (3.532097 - (wtd_gmea...
5,12,73.270300,abs(square((3.532097 - (wtd_gmean_ElectronAffi...
6,13,73.185326,abs(square(mean_FusionHeat - (3.532097 - (wtd_...
7,26,72.352270,square((((gmean_ElectronAffinity - exp(square(...
8,27,69.892760,square(((1.4489685 ^ wtd_std_ElectronAffinity)...


In [33]:
print(model.expressions_[5].iloc[8]["equation"])

square(((1.4489685 ^ wtd_std_ElectronAffinity) - ((gmean_ElectronAffinity - abs(exp(square(cos(range_fie) + 0.17120455)))) - exp(cos(exp(0.36404762 + cos(abs(range_fie))) - gmean_ElectronAffinity)))) * -0.7740606)


$$
\begin{aligned}
T_c = \biggl(
    & \left( 1.4489685^{\mathrm{wtd\_std\_ElectronAffinity}} \right) 
    - \biggl(
        \mathrm{gmean\_ElectronAffinity} 
        - \left| 
            \exp\left( 
                \left( \cos\left( \mathrm{range\_fie} \right) + 0.17120455 \right)^2 
            \right) 
        \right| \\
    & \quad - \exp\left( 
        \cos\left( 
            \exp\left( 
                0.36404762 + \cos\left( \left| \mathrm{range\_fie} \right| \right) 
            \right) - \mathrm{gmean\_ElectronAffinity} 
        \right) 
    \right)
    \biggr)
    \cdot (-0.7740606)
\biggr)^2
\end{aligned}
$$



#### **Cluster 7**

In [34]:
model.expressions_[6]

,complexity,loss,equation
0,1,48.286533,6.958004
1,3,40.226906,wtd_range_FusionHeat + 5.2950616
2,4,40.097640,cube(wtd_gmean_fie) + 6.435236
3,5,37.571583,(mean_fie + wtd_range_FusionHeat) + 5.1406293
4,6,36.294743,cube(wtd_gmean_fie) + (wtd_range_FusionHeat + ...
5,7,36.225110,abs((cube(wtd_gmean_fie) + wtd_range_FusionHea...
6,8,34.367886,exp(gmean_fie / 1.9086165) + (wtd_range_Fusion...
7,9,33.955875,(wtd_range_FusionHeat + exp(abs(gmean_fie) / 1...
8,10,31.169767,((wtd_range_FusionHeat + exp(gmean_fie - 1.908...


In [35]:
print(model.expressions_[6].iloc[8]["equation"])

((wtd_range_FusionHeat + exp(gmean_fie - 1.9086165)) + 2.5845776) * 1.2219173


$$
T_c = ((\mathrm{wtd\_range\_FusionHeat} + \exp{(\mathrm{gmean\_fie} - 1.9086165)}) + 2.5845776) \cdot 1.2219173
$$

#### **Cluster 8**

In [36]:
model.expressions_[7]

,complexity,loss,equation
0,1,684.47880,71.73812
1,3,513.01526,wtd_mean_Valence / -0.0121565135
2,5,499.76450,(wtd_mean_Valence / -0.0121565135) - -1.9581596
3,6,460.32544,square((wtd_std_Valence + -2.7054062) / 0.4051...
4,7,372.62207,((range_atomic_radius + 0.18340662) - wtd_std_...
5,8,365.31992,std_ThermalConductivity - ((wtd_std_Valence + ...
6,10,360.96270,square(((std_ThermalConductivity + range_atomi...
7,11,347.73044,square((std_ThermalConductivity + (range_atomi...
8,12,329.15277,square(((neg(range_atomic_radius) + wtd_std_Va...
9,16,321.13766,square(((std_ThermalConductivity + sin(range_a...


In [37]:
print(model.expressions_[7].iloc[15]["equation"])

square((((neg(sin(sin((range_atomic_radius + 2.902781) + (std_ThermalConductivity * wtd_mean_Valence)))) + wtd_std_Valence) - (wtd_mean_FusionHeat + (range_atomic_radius + range_atomic_radius))) / 0.47929877) + -3.7298758) - (entropy_Density - gmean_ElectronAffinity)


$$
\begin{aligned}
T_c = \biggl(
    \biggl(
        \frac{
            -\sin\left(
                \sin\left(
                    \left( \mathrm{range\_atomic\_radius} + 2.902781 \right)
                    + \left( \mathrm{std\_ThermalConductivity} \cdot \mathrm{wtd\_mean\_Valence} \right)
                \right)
            \right)
            + \mathrm{wtd\_std\_Valence}
            - \left(
                \mathrm{wtd\_mean\_FusionHeat} + \left( \mathrm{range\_atomic\_radius} + \mathrm{range\_atomic\_radius} \right)
            \right)
        }{
            0.47929877
        }
        - 3.7298758
    \biggr)^2 \\
    & \quad
    - \left( \mathrm{entropy\_Density} - \mathrm{gmean\_ElectronAffinity} \right)
\end{aligned}
$$


#### **Cluster 9**

In [38]:
model.expressions_[8]

,complexity,loss,equation
0,1,884.75555,68.274216
1,3,845.62210,wtd_gmean_Valence * -89.822014
2,4,711.16990,square(std_atomic_mass + 7.73254)
3,5,681.36830,(std_Density * 35.450253) + 60.38525
4,6,643.41766,square((std_Density + 7.3967357) + mean_Therma...
5,8,580.47710,square((std_Density / -0.45117334) + (-7.48904...
6,9,564.11250,square((exp(wtd_gmean_Valence) * -50.06931) + ...
7,10,535.89280,square(wtd_mean_ThermalConductivity - ((std_De...
8,14,512.39667,cube(1.9783194 - wtd_gmean_Valence) * (mean_Th...
9,17,465.17697,cube(0.4802176 - (wtd_gmean_Valence + -0.05339...


In [39]:
print(model.expressions_[8].iloc[11]["equation"])

((((range_atomic_radius + -1.0669534) - (wtd_gmean_Valence * (wtd_mean_ThermalConductivity - ((wtd_gmean_Valence * 3.5607712) - wtd_gmean_Valence)))) - -0.056062497) + (square(std_atomic_mass) + mean_ThermalConductivity)) * 22.776796


\begin{aligned}
T_c =\ 
& ((( 
    \mathrm{range\_atomic\_radius} - 1.0669534 ) \\
& \quad -\ \mathrm{wtd\_gmean\_Valence} \cdot (
    \mathrm{wtd\_mean\_ThermalConductivity} - (
        \mathrm{wtd\_gmean\_Valence} \cdot 3.5607712 
        - \mathrm{wtd\_gmean\_Valence} )
    )
) \\
& \quad +\ 0.056062497 ) \\
& \quad +\ \mathrm{std\_atomic\_mass}^2 \\
& \quad +\ \mathrm{mean\_ThermalConductivity} \cdot 22.776796
\end{aligned}


#### **Cluster 10**

In [40]:
model.expressions_[9]

,complexity,loss,equation
0,1,995.32460,56.540817
1,3,979.25024,mean_ThermalConductivity + 56.13125
2,4,821.51880,square(mean_ThermalConductivity + 7.12044)
3,5,623.01770,(0.7532081 - gmean_ElectronAffinity) / 0.02181...
4,6,617.50670,abs((0.7532081 - gmean_ElectronAffinity) / 0.0...
5,7,533.99680,cube((wtd_mean_ThermalConductivity + neg(gmean...
6,9,511.70834,cube((neg(gmean_ElectronAffinity) + wtd_mean_T...
7,11,485.55362,((wtd_std_ThermalConductivity - gmean_Electron...
8,12,483.96167,square((1.0157554 + (wtd_std_ThermalConductivi...
9,13,442.58405,square((((wtd_std_ThermalConductivity - gmean_...


In [41]:
print(model.expressions_[9].iloc[11]["equation"])

abs(((std_atomic_mass - wtd_std_ThermalConductivity) + -10.558848) - square(((-1.0472109 - (wtd_std_ThermalConductivity + 2.33174)) + 0.55530715) + ((gmean_ElectronAffinity - (wtd_std_ThermalConductivity + 0.26690817)) / abs(inv(std_atomic_mass + 1.523747)))))


\begin{aligned}
T_c =\ 
& \left| 
    \left( 
        \mathrm{std\_atomic\_mass} - \mathrm{wtd\_std\_ThermalConductivity} 
    \right) 
    + (-10.558848) \right. \\
& \quad 
\left. - 
    \left( 
        \left( 
            \left( 
                -1.0472109 - (\mathrm{wtd\_std\_ThermalConductivity} + 2.33174) 
            \right) 
            + 0.55530715 
        \right) 
        + 
        \left( 
            \frac{
                \mathrm{gmean\_ElectronAffinity} 
                - (\mathrm{wtd\_std\_ThermalConductivity} + 0.26690817)
            }{
                \left| 
                    \frac{1}{
                        \mathrm{std\_atomic\_mass} + 1.523747
                    } 
                \right|
            } 
        \right)^2
    \right) 
\right|
\end{aligned}


#### **Cluster 11**

In [42]:
model.expressions_[10]

,complexity,loss,equation
0,1,202.44516,11.762546
1,3,177.92763,12.702625 ^ wtd_std_fie
2,4,169.77472,exp(wtd_mean_atomic_mass - -2.8786032)
3,5,166.34001,square(exp(wtd_mean_atomic_mass - -1.6061039))
4,6,160.63231,cube(exp(exp(wtd_mean_atomic_mass))) - -3.1220164
5,7,151.79529,sqrt(square(-9.059844 / exp(sin(gmean_fie))))
6,9,147.94373,sqrt(square(12.433409 / exp(sin(gmean_fie + 0....
7,10,147.09420,square(sqrt(12.433409 / exp(sin(sin(gmean_fie ...
8,11,147.01599,square(sqrt(12.433409 / exp(sin(gmean_fie + 0....
9,12,145.55513,sqrt(square(12.433409 / exp(sin((gmean_fie - a...


In [43]:
print(model.expressions_[10].iloc[12]["equation"])

sqrt(square(12.433409 / exp(sin((gmean_fie - abs(sin(gmean_fie))) + 0.9722753)))) - range_ElectronAffinity


$$
T_c = \sqrt{\left(\frac{12.433409}{\exp{\sin{((\mathrm{gmean\_fie} - |\sin{\mathrm{gmean\_fie}}))}}}\right)^2} - \mathrm{range\_ElectronAffinity}
$$

#### **Cluster 12**

In [44]:
model.expressions_[11]

,complexity,loss,equation
0,1,1501.99490,87.04494
1,3,632.19684,range_atomic_radius * 115.20887
2,5,629.98676,(range_atomic_radius * 116.51489) + wtd_mean_a...
3,6,572.16480,(range_atomic_radius / 0.0107878195) / cos(wtd...
4,7,553.44696,abs((range_atomic_radius / 0.011350637) / cos(...
5,12,541.94556,range_atomic_radius * exp(sqrt(abs((wtd_gmean_...
6,13,529.26570,(20.261904 - inv(wtd_mean_atomic_radius)) + cu...
7,14,498.45660,range_atomic_radius * exp(sqrt((2.4043283 ^ sq...
8,15,479.38080,exp(sqrt(exp(exp(wtd_gmean_ElectronAffinity) ^...
9,18,474.92123,abs((exp(sqrt(exp(exp(wtd_gmean_ElectronAffini...


In [45]:
print(model.expressions_[11].iloc[14]["equation"])

(wtd_range_atomic_radius / -0.07890964) + (((exp(sqrt(exp(exp(wtd_gmean_ElectronAffinity) ^ std_atomic_mass) + ((wtd_gmean_ElectronAffinity + -3.0278127) / -0.18163086))) * range_atomic_radius) - 0.0024655748) - (wtd_mean_ThermalConductivity * 1.5270501))


$$
(\frac{\mathrm{wtd\_range\_atomic\_radius}}{-0.07890964}) + (((\exp{\sqrt{\exp{\exp{\mathrm{wtd\_gmean\_ElectronAffinity}^\mathrm{std\_atomic\_mass}} + (\frac{(\mathrm{wtd\_gmean\_ElectronAffinity} - -3.0278127)}{ -0.18163086})}} \cdot \mathrm{range\_atomic\_radius}} - 0.0024655748) - (\mathrm{wtd\_mean\_ThermalConductivity} \cdot 1.5270501)))
$$

#### **Cluster 13**

In [46]:
model.expressions_[12]

,complexity,loss,equation
0,1,514.80510,73.69545
1,3,452.23200,wtd_mean_fie * 82.70947
2,5,445.67654,(wtd_mean_fie * 128.25024) - 40.635643
3,6,443.58350,abs((wtd_mean_fie * 128.25024) + -41.737915)
4,9,443.44852,abs(abs((wtd_mean_fie * 128.25024) + -41.73791...
5,10,443.00220,abs(((wtd_mean_fie * 128.25024) + -41.737915) ...
6,12,337.72122,abs(81.99711 - (0.0071012434 ^ cos((wtd_mean_V...
7,14,312.96954,abs((0.015125314 ^ cos((wtd_mean_Valence / (wt...


In [47]:
print(model.expressions_[12].iloc[7]["equation"])

abs((0.015125314 ^ cos((wtd_mean_Valence / (wtd_gmean_fie ^ wtd_range_ThermalConductivity)) / square(wtd_range_fie))) - 81.40365)


$$
T_c = |((0.015125314^{\cos{(\frac{\mathrm{wtd\_mean\_Valence}}{(\mathrm{wtd\_gmean\_fie}^\mathrm{wtd\_range\_ThermalConductivity})})} \cdot \mathrm{wtd\_range\_fie}}) - 81.30365)|

#### **Cluster 14**

In [48]:
model.expressions_[13]

,complexity,loss,equation
0,1,683.73440,58.118122
1,3,578.86224,wtd_std_Valence * -67.29319
2,4,506.57285,cube(wtd_gmean_Valence * -4.7420697)
3,5,487.51456,cube(square(wtd_gmean_Valence * 2.4052083))
4,6,470.47577,cube(wtd_gmean_Valence * (wtd_mean_Valence / 0...
5,8,468.89795,cube(wtd_gmean_Valence * (wtd_mean_Valence / 0...
6,11,468.24620,cube(wtd_gmean_Valence * (wtd_mean_Valence / 0...
7,12,442.28635,square(cube((((cos(wtd_std_Valence) ^ 6.921070...
8,15,422.75280,square(cube(abs(((1.1799345 * wtd_gmean_Valenc...
9,17,422.11017,square(cube(abs(((1.1799345 * wtd_gmean_Valenc...


In [53]:
print(model.expressions_[13].iloc[9]["equation"])

square(cube(abs(((1.1799345 * wtd_gmean_Valence) - 1.0482824) + (cos(wtd_std_Valence) ^ 8.210996))) - mean_fie) - -0.5256394


$$
T_c = (|((1.1799345  \cdot \mathrm{wtd\_gmean\_Valence}) - 1.0482824) + \cos{\mathrm{wtd\_std\_Valence}^{8.210996}}|^3 - \mathrm{mean\_fie})^2 + 0.5256394
$$

#### **Cluster 15**

In [49]:
model.expressions_[14]

,complexity,loss,equation
0,1,660.87720,34.65458
1,3,394.76346,0.013902749 ^ wtd_mean_Valence
2,4,377.26065,cube(wtd_gmean_Valence) * -78.20767
3,5,357.59985,exp(neg(wtd_std_Valence + -2.931211))
4,7,357.59960,exp(neg(wtd_std_Valence + -1.7593584) + 1.1719...
5,8,320.73328,cube(wtd_entropy_atomic_mass + neg(cos(wtd_std...
6,10,311.49582,exp(neg((wtd_std_Valence + cos(wtd_entropy_ato...
7,12,309.26227,abs(exp(neg((wtd_std_Valence + -3.919459) + co...
8,14,288.59692,exp(abs(neg((wtd_std_Valence + cos((2.6621842 ...
9,15,230.02043,cube((wtd_entropy_atomic_mass + wtd_range_Dens...


In [54]:
print(model.expressions_[14].iloc[11]["equation"])

neg(neg(exp(neg((cos((exp(wtd_std_Valence + (wtd_range_Valence + 0.26946986)) / 0.2021761) + 1.6960412) + neg(exp(cos(exp(wtd_std_Valence) / gmean_Density)))) - 1.0277864))))


$$
T_c = \left(
    - \left(
        - \exp \left(
            - \left(
                \cos \left(
                    \frac{
                        \exp\left(
                            \mathrm{wtd\_std\_Valence} + \left(\mathrm{wtd\_range\_Valence} + 0.26946986\right)
                        \right)
                    }{0.2021761}
                    + 1.6960412
                \right)
                + 
                - \exp \left(
                    \cos \left(
                        \frac{
                            \exp \left(\mathrm{wtd\_std\_Valence}\right)
                        }{
                            \mathrm{gmean\_Density}
                        }
                    \right)
                \right)
                - 1.0277864
            \right)
        \right)
    \right)
\right)
$$


#### **Cluster 16**

In [50]:
model.expressions_[15]

,complexity,loss,equation
0,1,357.39282,21.076952
1,3,339.85960,21.076952 - wtd_gmean_ElectronAffinity
2,4,263.90118,24.231678 / exp(wtd_mean_Valence)
3,6,244.89635,34.300774 - abs(wtd_gmean_ElectronAffinity * -...
4,7,244.57104,34.981255 - abs(abs(wtd_gmean_ElectronAffinity...
5,8,241.28130,34.300774 - abs((wtd_gmean_ElectronAffinity - ...
6,9,240.22552,abs(34.300774 - abs((wtd_gmean_ElectronAffinit...
7,10,227.11543,abs(((1.335259 - wtd_mean_Valence) / 0.0474187...
8,12,222.45715,((2.399725 - wtd_mean_Valence) / 0.059883133) ...
9,14,220.01170,(((2.399725 - wtd_mean_Valence) / 0.059883133)...


In [55]:
print(model.expressions_[15].iloc[14]["equation"])

abs((((0.9801425 - wtd_mean_Valence) / -0.066473566) - wtd_mean_Valence) * (1.2201971 / abs(sqrt(sqrt(abs((mean_atomic_mass - ((mean_atomic_mass - wtd_mean_atomic_mass) * 0.5446983)) - -0.18084972)))))) + abs(wtd_std_ThermalConductivity)


$$
T_c = 
\left|
    \left(
        \frac{
            0.9801425 - \mathrm{wtd\_mean\_Valence}
        }{
            -0.066473566
        }
        - \mathrm{wtd\_mean\_Valence}
    \right)
    \cdot 
    \left(
        \frac{
            1.2201971
        }{
            \left|
                \sqrt{
                    \sqrt{
                        \left|
                            \left(
                                \mathrm{mean\_atomic\_mass} 
                                - \left(
                                    \left(
                                        \mathrm{mean\_atomic\_mass} 
                                        - \mathrm{wtd\_mean\_atomic\_mass}
                                    \right)
                                    \cdot 0.5446983
                                \right)
                            \right)
                            - (-0.18084972)
                        \right|
                    }
                }
            \right|
        }
    \right)
\right|
+ \left| \mathrm{wtd\_std\_ThermalConductivity} \right|
$$


#### **Cluster 17**

In [51]:
model.expressions_[16]

,complexity,loss,equation
0,1,1473.61730,50.051132
1,3,1336.15860,wtd_mean_ThermalConductivity * 38.12355
2,4,781.86950,square(wtd_mean_ThermalConductivity + 6.462124)
3,5,609.24460,(wtd_mean_ThermalConductivity * 24.543894) + 3...
4,6,608.86690,square(range_ThermalConductivity + (wtd_mean_T...
5,7,605.81450,((wtd_mean_ThermalConductivity * 24.543894) + ...
6,8,506.69630,square((std_atomic_mass + 4.547156) + (wtd_mea...
7,10,471.48285,square(((std_atomic_mass + 4.547156) + (wtd_me...
8,12,467.07416,square((std_atomic_mass + 4.547156) + ((wtd_me...
9,14,456.97754,square((((3.0679328 + wtd_std_atomic_mass) * (...


In [56]:
print(model.expressions_[16].iloc[11]["equation"])

square((((wtd_std_atomic_mass + 3.0679328) * ((0.43445778 - wtd_range_FusionHeat) - wtd_gmean_Valence)) - -1.3575011) + abs(abs(wtd_mean_ThermalConductivity))) - -1.6665066


$$
T_c = \left(
    \left(
        \left(
            \mathrm{wtd\_std\_atomic\_mass} + 3.0679328
        \right)
        \cdot
        \left(
            0.43445778 - \mathrm{wtd\_range\_FusionHeat} - \mathrm{wtd\_gmean\_Valence}
        \right)
        + 1.3575011
    \right)
    + \left| \left| \mathrm{wtd\_mean\_ThermalConductivity} \right| \right|
\right)^2
+ 1.6665066
$$


### 🤔 **Conclusions**

In general, the expressions created are highly complex. The simplest ones have a high loss function. Furthermore, in the future, the study of the equations created could be useful, in combination with explainable techniques, for a better understanding of the properties responsible for the phenomenon of superconductivity.